# run THDM-II model

The idea is run the model

In [1]:
import numpy as np
import pandas as pd
import subprocess
import time

%matplotlib inline
import matplotlib.pyplot as plt 

In [2]:
import pyslha
import pyTHDM_LesHouches_generator
import sys

Cambiar valores dentro del archivo LesHouches

In [61]:
#Open the file that will created the LesHouches file
xdict = pyTHDM_LesHouches_generator.buildSLHAinFile()

#Created a benchmark point                                                                            
'''
L1 = 0.249746/2.
L2 = 0.579478/2.
L3 = 0.0519579
L4 = 0.00285079
L5 = -0.336346
M12 = 1.0e-4
tb = 2.0701
'''
L1 = 0.121124
L2 = 0.288218
L3 = 0.049769
L4 = -0.0415422
L5 = -0.381272
M12 = -85113.8
tb = 2.0701

xdict.blocks['MINPAR'].entries[1]='%.6E    # Lambda1Input'%L1
xdict.blocks['MINPAR'].entries[2]='%.6E    # Lambda2Input'%L2
xdict.blocks['MINPAR'].entries[3]='%.6E    # Lambda3Input'%L3
xdict.blocks['MINPAR'].entries[4]='%.6E    # Lambda4Input'%L4
xdict.blocks['MINPAR'].entries[5]='%.6E    # Lambda5Input'%L5
xdict.blocks['MINPAR'].entries[9]='%.6E    # M12inputt'%M12
xdict.blocks['MINPAR'].entries[10]='%.6E    # TanBeta' %tb

#Write and created the Leshouches file generator
pyslha.writeSLHAFile('LesHouches.in.THDMII_low',xdict)  

#run SPheno
spheno = subprocess.getoutput('~/tmp/SPheno-4.0.4/bin/./SPhenoTHDMII LesHouches.in.THDMII_low')    
SPheno_output = subprocess.getoutput('cat SPheno.spc.THDMII')

#capturo algo de spheno

#MASAS
mh1 = eval(SPheno_output.split('#   PDG code')[1].split()[3])
mh2 = eval(SPheno_output.split('#   PDG code')[1].split()[7])
mAh = eval(SPheno_output.split('#   PDG code')[1].split()[11])
mHm = eval(SPheno_output.split('#   PDG code')[1].split()[15])
print("========== MASAS ==============")
print("mh1=",mh1,"mh2=",mh2,"mAh=",mAh,"mHm=",mHm)
print("===============================\n")

#Couplings
h1bb = eval(SPheno_output.split('HiggsBoundsInputHiggsCouplingsFermions')[1].split()[1])
h1tt = eval(SPheno_output.split('# h_1 s s coupling')[1].split()[0])
h1tautau = eval(SPheno_output.split('# h_1 c c coupling')[1].split()[0])
print("============ Couplings ================")
print("chbb=",h1bb,"chtt=",h1tt,"chtautau=",h1tautau)
print("============================\n")

#RUN Higgsbounds

#copio los archivos a la carpeta Input_files
subprocess.getoutput('cp Messages.out effC.dat BR* Key.dat LEP* LesHouches.in.THDMII_low MH* SPheno.spc.THDMII Input_Files/')
#run HiggsBounds
hb = subprocess.getoutput('~/Work/HiggsBounds-4.3.1/./HiggsBounds LandH effC 3 1 ~/Work/Documents_compartidos/THDM-II/Input_Files/')

#Capturo lo que necesito del archivo de resultados
HB_results = subprocess.getoutput('cat Input_Files/HiggsBounds_results.dat')
HBresult = eval(HB_results.split('cols:')[1].split()[15])
print("HBresult=",HBresult)


========== MASAS ==============
mh1= 123.596705 mh2= 482.466725 mAh= 490.048548 mHm= 479.554884

============ Couplings ================
chbb= 1.14377008 chtt= 0.963367851 chtautau= 1.14377008

HBresult= 1
